In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')
small_llm = ChatOpenAI(model='gpt-4o-mini')

### 1. RDBMS to VectorDB
- 변동성이 적은 Admin 메뉴 데이터 VeoctorDB 적재

In [3]:
import mysql.connector
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="1234",
    database="noticeboard"
)
cursor = conn.cursor(dictionary=True)

# 1. 데이터 조회
cursor.execute("""
    SELECT page_id, domain, page_url, description
    FROM page_metadata
""")
rows = cursor.fetchall()

# 2. LangChain 데이터 형식(Document)으로 변환
documents = []
for row in rows:
    # 벡터화할 핵심 텍스트: description
    page_content = row['description'] 
    
    # 저장할 메타데이터: 검색 결과에서 꺼내 쓸 정보들
    metadata = {
        "page_id": row["page_id"],
        "domain": row["domain"],
        "page_url": row["page_url"]
    }
    
    documents.append(Document(page_content=page_content, metadata=metadata))

# 3. Embedding 모델 설정 및 Vector DB 저장
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Chroma DB에 저장 - .from_documents() 메서드 사용
vector_db = Chroma(
    persist_directory="./my_vector_db",
    embedding_function=embeddings
)


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_9920\4035165175.py:40: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vector_db = Chroma(


In [5]:
# 사용자의 질문(의도) 시뮬레이션
query = "상품 업로드 어디서 해?" # "비밀번호를 잊어버렸는데 로그인 어떻게 해?"

# 유사도 검색 수행 (가장 유사한 2개 가져오기)
search_results = vector_db.similarity_search(query, k=2)

for doc in search_results:
    print(f"도메인: {doc.metadata['domain']}")
    print(f"경로(URL): {doc.metadata['page_url']}")
    print(f"매칭된 설명: {doc.page_content}")
    print("-" * 30)

도메인: 재고
경로(URL): /inventory
매칭된 설명: 상품 재고 현황을 조회하는 화면이다. 현재 수량, 입출고 내역을 확인하고 재고 관리를 수행한다.
------------------------------
도메인: 보안
경로(URL): /2fa/select
매칭된 설명: 2단계 인증 수단(앱, 이메일, QR 등)을 선택하는 화면이다.
------------------------------


### 2. RAG 답변 체인 구성

In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# 프롬프트 디자인 (페르소나 부여)
template = """
당신은 시스템 이용을 돕는 친절한 AI 가이드입니다. 
사용자의 질문에 대해 [검색 결과]를 바탕으로 자연스럽고 친절하게 답변하세요.
답변에는 반드시 이동해야 할 페이지의 이름과 URL을 포함해 주세요.

[검색 결과]:
{context}

사용자 질문: {question}

친절한 답변:"""

prompt = ChatPromptTemplate.from_template(template)

# 검색 및 답변 생성 체인 구성
def format_docs(docs):
    # 검색된 문서들을 하나의 텍스트로 합침
    return "\n\n".join([f'''페이지명: {d.metadata['domain']}
                        URL: {d.metadata['page_url']}\n설명: {d.page_content}''' for d in docs])

retriever = vector_db.as_retriever(search_kwargs={"k": 2})

chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
)

In [ ]:
response = chain.invoke("재고 보고싶어")
print(response.content)

"""
안녕하세요! 상품의 재고 현황과 입출고 내역을 확인하고 싶으시군요. 재고 관리를 위해서는 "재고" 페이지로 이동하시면 됩니다. 아래의 링크를 클릭하시면 자세한 정보를 확인하실 수 있습니다:

[재고 페이지](/inventory)

필요한 정보를 잘 찾으시길 바랍니다! 추가로 궁금한 점이 있으면 언제든지 말씀해 주세요.
"""

### 3. 도구 정의

In [8]:
from langchain_core.tools import tool

@tool
def search_page_tool(query: str) -> str:
    """관리자용 AI 어시스턴트가 메뉴 페이지를 검색하는 도구"""
    search_results = vector_db.similarity_search(query, k=2)
    formatted_results = format_docs(search_results)
    return formatted_results

In [ ]:
from langchain_core.tools import tool
import requests

@tool
def search_inventory_tool(item_name: str):
    """실시간 재고 시스템 API를 호출하여 최신 수량과 상세 페이지 경로를 가져오는 도구"""
    
    API_URL = f"http://localhost:8081/inventories/{item_name}"
    response = requests.get(API_URL)
    item_list = response.json()
    
    if isinstance(item_list, list) and len(item_list) > 0:
        results = []
        for item in item_list:
            # 데이터 추출 
            prod_id = item.get('productId', '알수없음')
            qty = item.get('orderQty', 0)
            price = item.get('unitPrice', 0)
            path = f"/inventory/{prod_id}"
            
            # 개별 항목 문자열 생성
            info = f"- 상품ID: {prod_id}\n  재고: {qty}개\n  단가: {price}원\n  경로: {path}"
            results.append(info)
        
        # 모든 결과를 줄바꿈으로 합쳐서 반환
        all_inventory_info = "\n\n".join(results)
        return f"'{item_name}' 검색 결과 총 {len(item_list)}건이 발견되었습니다:\n\n{all_inventory_info}"
        
    else:
        return f"현재 시스템상에 '{item_name}' 품목이 존재하지 않습니다."

In [10]:
result = search_inventory_tool("마우스")
print(result)

현재 시스템상에 '마우스' 품목이 존재하지 않습니다.


In [21]:
from langgraph.prebuilt import ToolNode

tool_list = [search_page_tool, search_inventory_tool]
llm_with_tools = small_llm.bind_tools(tool_list)
tool_node = ToolNode(tool_list)

### 상태 정의 및 그래프 생성

In [15]:
from typing_extensions import List, TypedDict
from langchain_core.documents import Document
from langgraph.graph import StateGraph

# State 정의
class AgentState(TypedDict):
    query: str
    context: List[Document]
    answer: str

# 그래프 선언
graph_builder = StateGraph(AgentState)

### 노드 정의

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

rewrite_prompt = PromptTemplate.from_template("""
당신은 재고 관리 시스템의 검색 전문가입니다.
사용자의 질문을 분석하여 '재고 검색 API'에 던질 최적의 검색 키워드를 3~4개 추출하세요.

[지침]
1. 사용자가 '컴퓨터'라고 하면 ['컴퓨터', '모니터', '노트북', '데스크탑']과 같이 연관 상품을 제안하세요.
2. 너무 포괄적인 단어보다는 실제 상품이 존재할 법한 구체적인 단어를 선택하세요.
3. 반드시 JSON 리스트 형식으로만 응답하세요. 예: ["키워드1", "키워드2"]

사용자 질문: {user_query}
""")

def keyword_expansion(state: AgentState):
    """
    사용자의 질문을 분석하여 검색에 최적화된 2개의 키워드를 추출하는 노드
    """
    
    query = state['query']
    rewrite_chain = rewrite_prompt | llm | StrOutputParser()
    response = rewrite_chain.invoke({'user_query': query})
    return {'query': response}

In [20]:
initial_state = {'query': "요리할 때 필요한 제품 없을까?"}
result = keyword_expansion(initial_state)
print(result)

{'query': '["주방용품", "프라이팬", "냄비", "칼"]'}


In [ ]:
def generate_answer(state: AgentState):
    last_message = state['messages'][-1]    # 도구의 응답값이 messages에 쌓임
    prompt = f"정보: {state['messages']}\n사용자 질문: {state['query']}\n답변해줘."
    response = small_llm.invoke(prompt)
    return {"messages": [response]}